In [1]:
%matplotlib inline

# import dependencies
import tika
tika.initVM()
from tika import parser

import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold

from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

import matplotlib
from matplotlib import pyplot as plt


# web resources used
# https://www.kaggle.com/itratrahman/nlp-tutorial-using-python
    # ideally parse pdf data into dataframe format for feature engineering?
# https://stackoverflow.com/questions/50985619/how-to-read-pdf-files-which-are-in-asian-languages-chinese-japanese-thai-etc - not that useful but oh well
# https://stackoverflow.com/questions/46389254/how-to-parse-text-extracted-from-pdf-file-with-delimiter-using-python - answer by Grijesh Chauhan


In [2]:
import re

In [3]:
parsed = parser.from_file('AAAI-19_Accepted_Papers.pdf')
metadata = parsed["metadata"]
content = parsed["content"]

In [4]:
# checked content, is a string type, looks good when printed but is a load of formatting when called
# re split at new lines and \xa0
split_at_nums = re.split(r'\s+(?=\d)|(?<=\d)\s+', content)
# split_at_nums = re.split("\d+:?", content)
# this isn't that great because it's splitting at any instance that says 3D
# extractedText = re.split('\n|\xa0', content) # parsedText = [''.join([*filter(str.isalnum, e)]) for e in extractedText]

In [5]:
entries = [re.sub('\xa0|\n|\xad', '', line) for line in split_at_nums]
len(entries)

1179

In [6]:
# https://blog.manash.me/configuring-stanford-parser-and-stanford-ner-tagger-with-nltk-in-python-on-windows-f685483c374a

import nltk
import os

# core nlp needs java 8 to run
import os
java_path = "C:\Program Files (x86)\Common Files\Oracle\Java\javapath\java.exe"
os.environ['JAVAHOME'] = java_path

# downloaded punkt separately because error text demanded it
nltk.download('punkt')
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize

# change the path according to my system
stanford_classifier = r"C:\Users\Qiao\Downloads\sdnlp\stanford-ner-2018-10-16\stanford-ner-2018-10-16\classifiers\english.all.3class.distsim.crf.ser.gz"
stanford_ner_path = r"C:\Users\Qiao\Downloads\sdnlp\stanford-ner-2018-10-16\stanford-ner-2018-10-16\stanford-ner.jar"


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Qiao\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
# Creating Tagger Object
# checking to see if it worked
# takes a long time to process entries line by line, will process single file then clean up
st = StanfordNERTagger(stanford_classifier, stanford_ner_path, encoding='utf-8')

text = entries[11]

tokenized_text = word_tokenize(text)
classified_text = st.tag(tokenized_text)

print(classified_text)

[('81', 'O'), (':', 'O'), ('PhoneMD', 'O'), (':', 'O'), ('Learning', 'O'), ('to', 'O'), ('Diagnose', 'O'), ('Parkinson', 'O'), ("'s", 'O'), ('Disease', 'O'), ('from', 'O'), ('Smartphone', 'ORGANIZATION'), ('Data', 'ORGANIZATION'), ('Patrick', 'ORGANIZATION'), ('Schwab', 'ORGANIZATION'), ('(', 'ORGANIZATION'), ('ETH', 'ORGANIZATION'), ('Zurich', 'ORGANIZATION'), (')', 'O'), ('*', 'O'), (';', 'O'), ('Walter', 'PERSON'), ('Karlen', 'PERSON'), ('(', 'O'), ('ETH', 'O'), ('Zurich', 'LOCATION'), (')', 'O')]


In [8]:
# clean up formatters in single file from parsed content
single_file = re.sub('\xa0|\n|\xad', '', content)

In [9]:
text = single_file

tokenized_text = word_tokenize(text)
classified_text = st.tag(tokenized_text)

In [10]:
df = pd.DataFrame(classified_text[92:]
                 , columns=['tokenized_text','tag']) # content after header
col_list = df.columns
#df.loc[(df[col_list] == 'O').any(axis=1)]

In [11]:
# looping to find each tag in tuples too long, I can't optimize this; using dataframes to make categories
raw_context = pd.DataFrame(df.loc[df['tag'] == 'O', 'tokenized_text'], index=None)
raw_persons = pd.DataFrame(df.loc[df['tag'] == 'PERSON', 'tokenized_text'], index=None)
raw_org = pd.DataFrame(df.loc[df['tag'] == 'ORGANIZATION', 'tokenized_text'], index=None)

In [12]:
orgs = raw_org['tokenized_text'].values.tolist()

In [23]:
temp_list = []
orgs_list = []
for item in orgs:
    if item not in "()*": # assuming vast majority of organizatins delimited by round brackets
        temp_list.append(item)
    else:
        orgs_list.append(' '.join(temp_list))
        temp_list = []
        pass

In [25]:
raw_persons

,tokenized_text
21,Jiaolong
22,Yang
32,Wei
33,Liang
41,Xin
42,Tong
64,Sandra
65,Zilles
134,Yu
135,Cheng
